## CUDA setup

In [ ]:
!sudo apt-get install -y --no-install-recommends libnvinfer7=7.0.0-1+cuda10.0 \
    libnvinfer-dev=7.0.0-1+cuda10.0 \
    libnvinfer-plugin7=7.0.0-1+cuda10.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libnvinfer-dev libnvinfer-plugin7 libnvinfer7
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 143 MB of archives.
After this operation, 520 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer7 7.0.0-1+cuda10.0 [69.6 MB]
Get:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-dev 7.0.0-1+cuda10.0 [71.6 MB]
Get:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  libnvinfer-plugin7 7.0.0-1+cuda10.0 [2,109 kB]
Fetched 143 MB in 3s (53.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <

#  Installing necessary libraries


In [ ]:
%tensorflow_version 1.x
#!pip install tensorflow==1.15.2
!pip install tensorflow-gpu==1.15.2
!pip install -q gpt-2-simple==0.7.2

TensorFlow 1.x selected.
     |████████████████████████████████| 410.9 MB 31 kB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=1a3deeedaf7a6b37740639e910abd4c4bbe33aeb95a91e2615518e16902590a6
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.


In [ ]:
import gpt_2_simple as gpt2
from datetime import datetime

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Verifying GPU

In [ ]:
!nvidia-smi

Thu Dec 23 20:43:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11559251388955632759
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1260313731705644774
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 922248498910722416
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15964005991
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7960961174839188233
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


## Mounting Google Drive

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
main_path = "drive/MyDrive/shared/GPT/"
repo_dir = "eclipse_netbeans/"
project_name = "eclipse"
fold_id = 0
setting = "dynamic"
tests_path = main_path + "tests/" + repo_dir

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name=f'fold{fold_id}')

## Generating candidates using GPT-2

In [ ]:
!cp -R drive/MyDrive/shared/GPT/gpt gpt

In [ ]:
from gpt.gpt_manager_full import GPTManager

In [ ]:
gpt_manager = GPTManager()

In [ ]:
gpt_manager.java_model = gpt_manager.load_model('checkpoint/', 'run1')

244 end tokens.
Loading model checkpoint/run1/model-50000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-50000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
import json

def readTests(projectName, foldId):
    oneArgTests = []
    with open(f"{tests_path}{projectName}_ArgRecTests_fold{foldId}.txt") as f:
        lines = f.read().split('\n')
        for line in lines[:-1]:
            oneArgTests.append(json.loads(line))
        lines = None
    return oneArgTests

In [ ]:
def toSingleArgRecTest(this):
    test = {}
    test['filePath'] = this['filePath']
    test['numArg'] = 1 if this['argPos'] != 0 else 0
    test['lex_context'] = this['lex_context']
    test['excode_context'] = this['excode_context']
    test['next_excode'] = [this['next_excode']]
    test['next_lex'] = [this['next_lex']]
    test['expected_excode'] = this['expected_excode']
    test['expected_lex'] = this['expected_lex']
    test['ignored'] = this['ignored']
    test['argRecTestList'] = [this]
    test['id'] = this['test_id']
    return test

In [ ]:
def allTestsToSingleArgRecTest(oneArgTests):
    tests = []
    for i in range(len(oneArgTests)):
        test = oneArgTests[i]
        # SKIP METHOD INVOCATIONS WITH NO ARGUMENT PASSED
        if test['argPos'] > 0:
            test = toSingleArgRecTest(test)
            tests.append(test)
    return tests

In [ ]:
from gpt import preprocessor

def preprocess(target):
    target = preprocessor.empty_string_literal(target)
    target = preprocessor.remove_array_access_index(target)
    return target

In [ ]:
def matchesArg(expectedLex, result):
    if result == expectedLex:
        return True

    if '->' in expectedLex and '->' in result:
        return True

    if '.this' in expectedLex:
        if matchesArg(expectedLex[expectedLex.index('.this')+1:], result):
            return True

    if '.this' in result:
        if matchesArg(expectedLex, result[result.index('.this')+1:]):
            return True

    if expectedLex.startswith('this.'):
        if matchesArg(expectedLex[5:], result):
            return True

    if result.startswith('this.'):
        if matchesArg(expectedLex, result[5:]):
            return True

    return False

In [ ]:
def canAcceptGeneratedLexes(test):
    test = test['argRecTestList'][0]

    expectedLex = test['expected_lex']

    expectedLex = preprocess(expectedLex)
    if '{' in expectedLex:
        expectedLex = expectedLex[:expectedLex.index('{')].rstrip()

    for group in test['next_lex']:
        for candidate in group:
            candidate = preprocess(candidate)
            if matchesArg(expectedLex, candidate):
                return True

            alternateLex = None
            if 'methodAccessLex' in test:
                alternateLex = test['methodAccessLex']
            if 'objectCreationLex' in test:
                alternateLex = test['objectCreationLex']
            if alternateLex is not None and matchesArg(alternateLex, candidate):
                return True

            if 'staticMemberAccessLex' in test:
                if matchesArg(test['staticMemberAccessLex'], candidate):
                    return True
    return False

In [ ]:
expressionTypes = ['NAME', 'METHOD_INVOC', 'FIELD_ACCESS', 'ARRAY_ACCESS', 'CAST', 'STRING_LIT', 'NUM_LIT', 'CHAR_LIT', 'TYPE_LIT', 'BOOL_LIT',
    'NULL_LIT', 'OBJ_CREATION', 'ARR_CREATION', 'THIS', 'SUPER', 'COMPOUND', 'LAMBDA', 'METHOD_REF']
expressionTypeDict = {}

for i in range(len(expressionTypes)):
    expressionTypeDict[expressionTypes[i]] = i

tops = [1, 3, 5, 10]

In [ ]:
def validateTest(test):
    if not test['ignored']:
        adequateGeneratedLex = False
        if canAcceptGeneratedLexes(test):
            adequateGeneratedLex = True;
        if adequateGeneratedLex:
            return True
    return False

In [ ]:
import os
import shutil
from collections import defaultdict
from tqdm.notebook import tqdm as tqdm

os.makedirs(f'predictions/{repo_dir}gpt2/', exist_ok=True)

predictionFilePath = "predictions/" + repo_dir + "gpt2/" + f"{project_name}_ArgRecs_fold{fold_id}.txt"
prediction_details = []
tests = allTestsToSingleArgRecTest(readTests(project_name, fold_id))
cnt = -1

if not os.path.isfile(main_path + predictionFilePath):
    startId = 0
else:
    with open(main_path + predictionFilePath) as f:
        content = f.read()
        startId = content.count('\n')
        print(startId)
        lines = content.split('\n')
        totalRuntime = 0
        for line in lines[:-1]:
            prediction = json.loads(line)
            totalRuntime += prediction['runtime']

filePath = f"{main_path}predictions/{repo_dir}/filter_{setting}/{project_name}/fold{fold_id}/{project_name}_prediction_detail_flute_sequence.txt"
filter_preds = []
with open(filePath) as f:
    lines = f.read().split('\n')
    for line in lines[:-1]:
        filter_pred = json.loads(line)
        if filter_pred['answer'] != ')':
            filter_preds.append(filter_pred)

15817


In [ ]:
for i in tqdm(range(startId, len(tests))):
    test = tests[i]
    if test['numArg'] == 0:
        continue

    response = gpt_manager.generate_param(test)
    gptResults = response['result']

    prediction_detail = {}
    prediction_detail['test_id'] = test['id']
    prediction_detail['predictions'] = gptResults
    prediction_detail['answer'] = test['expected_lex']
    prediction_detail['arg_type'] = test['argRecTestList'][0]['argType']
    prediction_detail['runtime'] = response['runtime']
    prediction_detail['ignored'] = test['ignored']
    prediction_detail['sufficient_candidates'] = validateTest(test)
    #prediction_details.append(prediction_detail)

    with open(main_path + predictionFilePath, "a") as f:
        f.write(json.dumps(prediction_detail) + '\n')

  0%|          | 0/15047 [00:00<?, ?it/s]

/content/gpt/gpt_manager_full.py:580: RuntimeWarning: invalid value encountered in log
  log_prob = np.log(prob)


In [ ]:
# with open(predictionFilePath, "w") as f:
#     for prediction_detail in prediction_details:
#         f.write(json.dumps(prediction_detail) + '\n')

# shutil.copyfile(predictionFilePath, main_path + predictionFilePath)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1